####  In this Notebook we are going to scrape the data from wikepedia page

Lets first install required libraraies

In [71]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge scikit-learn --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be DOWNGRADED:

    blas:         1.1-openblas                          conda-forge --> 1.0-mkl              
    numpy:        1.13.3-py35_blas_openblas_201         conda-forge [blas_openblas] --> 1.13.3-py35ha266831_3
    scikit-learn: 0.20.0-py35_blas_openblasha84fab4_201 conda-forge [blas_openblas] --> 0.19.1-py35hbf1f462_0
    scipy:        1.1.0-py35_blas_openblash7943236_201  conda-forge [blas_openblas] --> 1.0.0-py35hcbbe4a2_0 

numpy-1.13.3-p 100% |################################| Time: 0:00:00  22.82 MB/s
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
lxml                      4.2.5            py35hc9114bc_0    conda-forge
Fetching package metadata .............
Solvin

##### Lets import the BeautifulSoup ,requests and pandas library

In [72]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd


Lets use the requests object to read the wikipedia page.
Pass the requests object to BeautifulSoup method (lxml parsing is used to parse the html page) which will retunr the soup object.
Using find_all method we can find the table. Use pandas read_html method to convert html table to list and then data frame.

In [73]:
res= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df_list = pd.read_html(str(table))
df=pd.DataFrame({'Postal Code':df_list[0][0],'Borough':df_list[0][1],'Neighborhood':df_list[0][2]})
df=df.drop([0],axis=0).reset_index(drop=True)

Remove the rows where column value of Borough is 'Not assigned'

In [74]:
df=df[df.Borough != 'Not assigned'].reset_index(drop=True)

1. Groupby the values based on 'Postcode' and 'Borough' and Concat the values of 'Neighborhood' in single row
2. Also Assign the value of'Borough' where 'Neighborhood' value is 'Not assigned'

In [75]:
df1=df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join)
df_postcode=pd.DataFrame(df1).reset_index(drop=False)
for index,row in df_postcode.iterrows():
   if row['Neighborhood']== 'Not assigned' :
     row['Neighborhood'] = row['Borough']
df_postcode.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [76]:
df_postcode.shape[0]

103

##### Longitude and Latitude

Get the longitude and latutude for each neighborhood using given csv file

In [77]:
df_lanlat=pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df_lanlat.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the data frames so that we will get final dataframes with all required columns including log and lat

In [78]:
df_p = pd.merge(df_postcode, df_lanlat, how='inner')
df_p.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Create a map of Toronto with neighborhoods superimposed on top.

In [79]:
!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors


address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))



Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be DOWNGRADED:

    blas:         1.1-openblas                          conda-forge --> 1.0-mkl              
    numpy:        1.13.3-py35_blas_openblas_201         conda-forge [blas_openblas] --> 1.13.3-py35ha266831_3
    scikit-learn: 0.20.0-py35_blas_openblasha84fab4_201 conda-forge [blas_openblas] --> 0.19.1-py35hbf1f462_0
    scipy:        1.1.0-py35_blas_openblash7943236_201  conda-forge [blas_openblas] --> 1.0.0-py35hcbbe4a2_0 

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [80]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_p['Latitude'], df_p['Longitude'], df_p['Borough'], df_p['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Lets work with only boroughs that contain the word Toronto

In [83]:
toronto_boroughs = df_p[df_p['Borough'].str.contains('Toronto')].reset_index(drop=False)
toronto_boroughs.head()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Lets explore Central Toronto Borough

In [84]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Borough'], toronto_boroughs['Neighborhood']):
    if borough=='Central Toronto':
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

#### Explore Neighborhoods in Central Toronto

In [113]:
Central_Toronto=toronto_boroughs[toronto_boroughs['Borough']=='Central Toronto'].reset_index(drop=True)
Central_Toronto.head()

CLIENT_ID = 'H55E5BKXI4F0YD0TDDZSBNYWWELW1VZZS1QAHO3EBPQZEKYF' # your Foursquare ID
CLIENT_SECRET = 'EC0X5VYQUZ0B45CZAKRKO4MP0FAZ23NOE5MY43MGDZLKSX01' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Central_Toronto_venues = getNearbyVenues(names=Central_Toronto['Neighborhood'],
                                   latitudes=Central_Toronto['Latitude'],
                                   longitudes=Central_Toronto['Longitude']
                                  )
Central_Toronto_venues.head()

Your credentails:
CLIENT_ID: H55E5BKXI4F0YD0TDDZSBNYWWELW1VZZS1QAHO3EBPQZEKYF
CLIENT_SECRET:EC0X5VYQUZ0B45CZAKRKO4MP0FAZ23NOE5MY43MGDZLKSX01
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lake,43.727910,-79.386857,Lake
1,Lawrence Park,43.728020,-79.388790,Rue Pigalle HC,43.727183,-79.387874,Jewelry Store
2,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
3,Davisville North,43.712751,-79.390197,Bob's Garden Centre,43.712872,-79.391707,Flower Shop
4,Davisville North,43.712751,-79.390197,Circle K,43.712834,-79.391554,Grocery Store


Apply one hot encoding to venue details

In [114]:
# one hot encoding Central_Toronto_venues
Central_Toronto_onehot = pd.get_dummies(Central_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Central_Toronto_onehot['Neighborhood'] = Central_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Central_Toronto_onehot.columns[-1]] + list(Central_Toronto_onehot.columns[:-1])
Central_Toronto_onehot = Central_Toronto_onehot[fixed_columns]

Central_Toronto_onehot.head()

,Neighborhood,Breakfast Spot,Café,Coffee Shop,Convenience Store,Dessert Shop,Flower Shop,Grocery Store,Health & Beauty Service,Indian Restaurant,Italian Restaurant,Jewelry Store,Lake,Park,Pizza Place,Roof Deck,Seafood Restaurant,Supermarket,Sushi Restaurant,Toy / Game Store
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Davisville North,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
import numpy as np

Central_Toronto_onehot.shape
Central_Toronto_grouped = Central_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Central_Toronto_grouped.head()
Central_Toronto_grouped.shape

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Central_Toronto_grouped['Neighborhood']

for ind in np.arange(Central_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Central_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Italian Restaurant,Café,Coffee Shop,Dessert Shop,Sushi Restaurant,Indian Restaurant,Toy / Game Store,Pizza Place,Seafood Restaurant,Lake
1,Davisville North,Breakfast Spot,Flower Shop,Grocery Store,Indian Restaurant,Café,Coffee Shop,Convenience Store,Dessert Shop,Health & Beauty Service,Toy / Game Store
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Supermarket,Roof Deck,Coffee Shop,Convenience Store,Toy / Game Store,Health & Beauty Service,Café,Dessert Shop,Flower Shop,Grocery Store
3,Lawrence Park,Lake,Jewelry Store,Toy / Game Store,Health & Beauty Service,Café,Coffee Shop,Convenience Store,Dessert Shop,Flower Shop,Grocery Store
4,"Moore Park,Summerhill East",Park,Toy / Game Store,Indian Restaurant,Café,Coffee Shop,Convenience Store,Dessert Shop,Flower Shop,Grocery Store,Health & Beauty Service
5,Roselawn,Health & Beauty Service,Toy / Game Store,Indian Restaurant,Café,Coffee Shop,Convenience Store,Dessert Shop,Flower Shop,Grocery Store,Italian Restaurant
